# IG² Attribution Rankings

This notebook visualizes the top features identified by IG² (Integrated Gradients squared) attribution.

**Purpose:**
- Identify the most important features for bias prediction per demographic
- Compare feature importance across demographics
- Understand which features have the highest attribution scores

**Input Data:**
- IG² attribution scores: [100,000] per demographic

**Output:**
- 3×3 grid of ranked bar charts (top-20 per demographic)
- Score distribution histograms

In [ ]:
import os
import sys
from pathlib import Path

import torch
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

PROJECT_ROOT = Path(os.getcwd()).parent.parent
sys.path.append(str(PROJECT_ROOT))

from src.visualization import (
    setup_korean_font,
    load_demographics,
    load_ig2_results,
    plot_ig2_rankings,
    get_demographic_labels
)

print(f"Project root: {PROJECT_ROOT}")

In [ ]:
# Setup
setup_korean_font()
sns.set_style('whitegrid')
sns.set_context('paper')

In [ ]:
# Configuration
DATA_DIR = PROJECT_ROOT / "data"
RESULTS_DIR = PROJECT_ROOT / "results"
ASSETS_DIR = PROJECT_ROOT / "notebooks" / "visualizations" / "assets"
ASSETS_DIR.mkdir(exist_ok=True, parents=True)

# Stage: 'pilot', 'medium', or 'full'
STAGE = "pilot"

# SAE configuration (for reference)
SAE_TYPE = "gated"  # 'standard' or 'gated'
LAYER_QUANTILE = "q2"  # 'q1', 'q2', or 'q3'

# Visualization parameters
TOP_K = 20

print(f"Stage: {STAGE}")
print(f"SAE type: {SAE_TYPE}")
print(f"Layer quantile: {LAYER_QUANTILE}")
print(f"Top-K: {TOP_K}")

## Load Data

In [ ]:
# Load demographics
demographics_dict = load_demographics(DATA_DIR)
demographic_labels_ko, demographic_labels_en = get_demographic_labels(demographics_dict)

# Load IG² results (loads from all per-demographic directories)
ig2_results = load_ig2_results(RESULTS_DIR, stage=STAGE)

print(f"Loaded IG² results for {len(ig2_results)} demographics")
for demo, data in ig2_results.items():
    print(f"  - {demo}")

## Plot Top-K Rankings

In [ ]:
fig = plot_ig2_rankings(
    ig2_results=ig2_results,
    demographic_labels_ko=demographic_labels_ko,
    demographic_labels_en=demographic_labels_en,
    save_path=ASSETS_DIR / f"ig2_rankings_{STAGE}_top{TOP_K}.png",
    top_k=TOP_K,
    figsize=(20, 15)
)

plt.show()

## Score Distribution Analysis

In [ ]:
# Plot score distributions
fig, axes = plt.subplots(3, 3, figsize=(18, 12))
axes = axes.flatten()

for i, (demo_ko, demo_en) in enumerate(zip(demographic_labels_ko, demographic_labels_en)):
    ax = axes[i]
    
    scores = ig2_results[demo_ko]
    if isinstance(scores, dict):
        scores = scores['feature_scores']
    
    scores = scores.cpu().numpy()
    
    # Histogram
    ax.hist(scores, bins=50, color='steelblue', alpha=0.7, edgecolor='black')
    ax.set_xlabel('IG² Score', fontsize=11)
    ax.set_ylabel('Frequency', fontsize=11)
    ax.set_title(f"{demo_ko} ({demo_en})", fontsize=13)
    ax.set_yscale('log')
    ax.grid(alpha=0.3)

plt.tight_layout()
plt.savefig(ASSETS_DIR / f"ig2_score_distributions_{STAGE}.png", dpi=300, bbox_inches='tight')
plt.show()

## Comparative Statistics

In [ ]:
import pandas as pd

# Compute statistics per demographic
stats = []

for demo_ko in demographic_labels_ko:
    scores = ig2_results[demo_ko]
    if isinstance(scores, dict):
        scores = scores['feature_scores']
    
    scores = scores.cpu().numpy()
    
    stats.append({
        'Demographic': demo_ko,
        'Max Score': scores.max(),
        'Mean Score': scores.mean(),
        'Median Score': np.median(scores),
        'Std Dev': scores.std(),
        'Top-10 Mean': np.sort(scores)[-10:].mean(),
        'Non-zero %': (scores > 0).mean() * 100
    })

df_stats = pd.DataFrame(stats)
print("\nIG² Score Statistics by Demographic:")
print("=" * 80)
print(df_stats.to_string(index=False))

# Save to CSV
df_stats.to_csv(ASSETS_DIR / f"ig2_statistics_{STAGE}.csv", index=False)
print(f"\nSaved to {ASSETS_DIR / f'ig2_statistics_{STAGE}.csv'}")

## Cross-Demographic Comparison

In [ ]:
# Compare top scores across demographics
fig, ax = plt.subplots(figsize=(12, 6))

x = np.arange(len(demographic_labels_ko))
max_scores = [ig2_results[d]['feature_scores'].max().item() if isinstance(ig2_results[d], dict) 
              else ig2_results[d].max().item() for d in demographic_labels_ko]

ax.bar(x, max_scores, color='coral', alpha=0.7, edgecolor='black')
ax.set_xticks(x)
ax.set_xticklabels(demographic_labels_ko, rotation=45, ha='right')
ax.set_ylabel('Max IG² Score', fontsize=13)
ax.set_title('최대 IG² 점수 비교\nMax IG² Score Comparison', fontsize=16, pad=15)
ax.grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.savefig(ASSETS_DIR / f"ig2_max_scores_comparison_{STAGE}.png", dpi=300, bbox_inches='tight')
plt.show()

## Interpretation

### What to Look For:

1. **Score Magnitude:**
   - Which demographics have the highest attribution scores?
   - Are scores concentrated in a few features or distributed?

2. **Top Features:**
   - Which features consistently appear at the top?
   - Are there demographic-specific vs. shared top features?

3. **Score Distribution:**
   - Is the distribution power-law (few high, many low)?
   - What percentage of features have non-zero scores?

### Next Steps:

1. Examine activation patterns for top features
2. Test suppression effects on highest-scoring features
3. Investigate feature overlap with UMAP clustering